In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, datasets
from tensorflow.keras.layers import LeakyReLU,PReLU
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

import os
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:
MEMORY_LIMIT = 4500
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=MEMORY_LIMIT)])
        print('Using GPU')
    except RuntimeError as e:
        print(e)

Using GPU


In [3]:
(X_train, Y_train), (X_test, Y_test) = datasets.cifar10.load_data()

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(72500, 32, 32, 3) (72500, 1)
(14500, 32, 32, 3) (14500, 1)


In [4]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [5]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [6]:
def build_model(dropout, neuronPct, neuronShrink):

    neuronCount = int(neuronPct * 5120)
    print(neuronCount)

    model = keras.Sequential()
    model.add( keras.Input(shape=X_train.shape[1:] ))
    layer = 0
    while neuronCount>20 and layer<20:

        model.add(layers.Conv2D(neuronCount, (3,3))) 
        model.add(layers.Conv2D(neuronCount, (3,3)))
        model.add( layers.BatchNormalization() ) 
        model.add( layers.Activation(PReLU()))
        
        neuronCount = neuronCount * neuronShrink
        layer += 1

    model.add(layers.Flatten())
    model.add(layers.Dense(512, kernel_initializer='he_uniform', activation=PReLU()))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(256, kernel_initializer='he_uniform', activation='relu'))
    model.add(layers.Dropout(dropout/2))
    model.add(layers.Dense(128, kernel_initializer='he_uniform', activation='relu'))
    model.add(layers.Dense(29, activation='softmax'))
    
    return model

In [7]:
model = build_model(dropout=0.2, neuronPct=0.1, neuronShrink=0.25)
model.summary()

512
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 512)       14336     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 512)       2359808   
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 512)       2048      
_________________________________________________________________
activation (Activation)      (None, 28, 28, 512)       401408    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 128)       589952    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)    

In [8]:
def evaluate_network(dropout,lr,neuronPct,neuronShrink):
    SPLITS = 2

    # Bootstrap
    boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)

    # Track progress
    mean_benchmark = []
    epochs_needed = []
    num = 0
    

    # Loop through samples
    for train, test in boot.split(X_train,Y_train):
        start_time = time.time()
        num+=1

        # Split train and test
        x_train = X_train[train]
        y_train = Y_train[train]
        x_test = X_train[test]
        y_test = Y_train[test]

        model = build_model(dropout, neuronPct, neuronShrink)
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(lr=lr),
                      metrics=['accuracy'])
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=3, verbose=1, mode='auto', restore_best_weights=True)

        # Train on the bootstrap sample
        model.fit(x_train, y_train, 
                  validation_data=(x_test,y_test),
                  batch_size=128,
                  callbacks=[monitor],
                  verbose=1,
                  epochs=10
                 )
        epochs = monitor.stopped_epoch
        epochs_needed.append(epochs)

        # Predict on the out of boot (validation)
        pred = model.predict(x_test)

        # Measure this bootstrap's log loss
        y_compare = np.argmax(y_test,axis=1) # For log loss calculation
        score = metrics.log_loss(y_compare, pred)
        mean_benchmark.append(score)
        m1 = statistics.mean(mean_benchmark)
        m2 = statistics.mean(epochs_needed)
        mdev = statistics.pstdev(mean_benchmark)

        # Record this iteration
        time_took = time.time() - start_time
        
    tensorflow.keras.backend.clear_session()
    return (-m1)

In [9]:
print(evaluate_network(
    dropout=0.2,
    lr=1e-3,
    neuronPct=0.2,
    neuronShrink=0.2))

1024
Epoch 1/10
476/510 [===========================>..] - ETA: 54s - loss: 3.3371 - accuracy: 0.0845

KeyboardInterrupt: 